In [1]:
import os
from datetime import date
from fpdf import FPDF

In [75]:
class PDF(FPDF):    
    def lines(self):
        self.set_line_width(0.0)
        self.line(5.0,5.0,205.0,5.0) # top one
        self.line(5.0,292.0,205.0,292.0) # bottom one
        self.line(5.0,5.0,5.0,292.0) # left one
        self.line(205.0,5.0,205.0,292.0) # right one
    
    def custom_header(self, logo, txt1, txt2=None):
        # Log
        self.image(logo, 10, 8, 33)
        # Arial bold 
        self.set_font('Arial', 'B', 30)
        # Move to the right
        self.cell(80)
        # Title
        self.cell(w=30, h=10, txt=txt1, border=0, ln=0, align='C')
        if txt2 != None:
            # Arial bold 15
            self.ln(20)
            self.cell(80)
            self.set_font('Arial', 'B', 20)
            self.cell(w=30, h=5, txt=txt2, border=0, ln=0, align='C')
        # Line break
        self.ln(20)
    
    def custom_footer(self, txt):
        # Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Position at 1.5 cm from bottom
        self.set_y(-30)
        # add text
        self.cell(w=0, h=0, txt=txt, border=0, ln=2, align='C') 
        # Date
        today = date.today()
        today = today.strftime("%d/%m/%Y")
        self.cell(w=5, h=8, txt=str(today) , border=0, ln=0, align='L')
        # Page number
        self.cell(w=180, h=8, txt='Page ' + str(self.page_no()), border=0, ln=2, align='R')
                
    
    def info_box(self, txt):
        # set font
        self.set_font('Arial', 'I', 10)
        #set box color
        self.set_fill_color(160,214,190)
        # add texte box info
        self.multi_cell(w=190, h=5, txt=txt , border=1, align='L', fill=True)
    
    def info_box_clust(self, txt):
        self.ln(140)
        # set font
        self.set_font('Arial', 'I', 6)
        #set box color
        self.set_fill_color(160,214,190)
        # add text box info
        self.multi_cell(w=80, h=5, txt=txt , border=1, align='L', fill=True)
    
    def disclaimer_box(self, txt):
        # set font
        self.set_font('Arial', 'I', 6)
        #set box color
        self.set_fill_color(240,128,128)
        # add texte box info
        self.multi_cell(w=190, h=5, txt=txt , border=1, align='L', fill=True)
        
    def subtitle_inflat(self):
        # Arial bold 15
        self.set_font('Arial', 'B', 20)
        # Title
        self.cell(w=10, h=80, txt='Overview clusters on inflated brain', border=0, ln=2, align='L')

        
    def imagey(self,im, y):
        pdf.image(im, 5, y, link='', type='', w=190, h=297/3)

In [76]:
subj_dir = '/home/mathilde/Documents/projects/MELD_classifier/process/220120_meld_tutorial/input/MELD_H4_3T_FCD_0001/predictions/reports'

In [77]:
pdf = PDF()#pdf object
pdf=PDF(orientation='P') # landscape
pdf=PDF(unit='mm') #unit of measurement
pdf=PDF(format='A4') #page format. A4 is the default value of the format, you don't have to specify it.

#initiate constante
pdf_w=210  #dim A4
pdf_h=297  #dim A4

logo = '/home/mathilde/Documents/other/MELD_logo.png'

temp_txt = 'blablbablaba'
text_info_1 = "Information: \n The MRI data of this patient has been processed through the MELD surface-based FCD detection algorithm. \n Page 1 of this report will show all detected clusters on an inflated view of the brain. \n Each subsequent page is an individual cluster"

text_info_2 = "For each cluster, the information below are provided: \n   -The hemisphere the cluster is on \n   -The surface area of the cluster (across the cortical surface) \n   -The location of the cluster \n   -The z-scores of the patient cortical features averaged within the cluster. \n   -The saliency of each feature to the network - if a feature is brighter pink, that feature was more important to the network. \n \n For more information, please read the Guide to using the MELD surface-based FCD detection."

disclaimer = "Disclaimer: The MELD surface-based FCD detection algorithm is intended for research purposes only and has not been reviewed or approved by the Medicines and Healthcare products Regulatory Agency (MHRA),European Medicine Agency (EMA) or by any other agency. Any clinical application of the software is at the sole risk of the party engaged in such application. \nThere is no warranty of any kind that the software will produce useful results in any way. Use of the software is at the recipient own risk" 

footer_txt="This report was created by Mathilde Ripart, Hannah Spitzer, Sophie Adler and Konrad Wagstyl on behalf of the MELD Project"


subject = 'MELD_H4_3T_FCD_0001'

In [78]:
today = date.today()

#### create main page with overview on inflated brain

#add page
pdf.add_page()
#add line contours
pdf.lines()
#add header
pdf.custom_header(logo, txt1='MELD report', txt2=f'Patient ID: {subject}')
#add info box
pdf.info_box(text_info_1)
#add disclaimer box
pdf.disclaimer_box(disclaimer)
#add image
pdf.subtitle_inflat()
im_inflat= os.path.join(subj_dir, f'inflatbrain_{subject}.png')
pdf.imagey(im_inflat,130)
#add footer date
pdf.custom_footer(footer_txt)

clusters=[1]
#### Create page for each cluster with MRI view and saliencies
for cluster in clusters: 
    #add page
    pdf.add_page()
    #add line contours
    pdf.lines()
    #add header
    pdf.custom_header(logo, txt1='MRI view & saliencies', txt2=f'cluster {cluster}')
    #add image
    im_mri= os.path.join(subj_dir, f'mri_{subject}_rh_c{cluster}.png')
    pdf.imagey(im_mri, 50)
    #add image
    im_sal= os.path.join(subj_dir, f'saliency_{subject}_rh_c{cluster}.png')
    pdf.imagey(im_sal, 150)
    #add txt
    pdf.info_box_clust(text_info_2)
    #add footer date
    pdf.custom_footer(footer_txt)





In [79]:
#save pdf
pdf.output(os.path.join(subj_dir,'test_pdf.pdf'),'F')

''